In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feedback or issues?
For any feedback or questions, please open an [issue](https://github.com/googleapis/python-aiplatform/issues).

# Vertex SDK for Python: Big Query Custom Container Training Example
To use this Jupyter notebook, copy the notebook to a Google Cloud Notebooks instance and open it. You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Jupyter automatically displays the return value of the last line in each cell. For more information about running notebooks in Google Cloud Notebook, see the [Google Cloud Notebook guide](https://cloud.google.com/vertex-ai/docs/general/notebooks).

This notebook demonstrate how to create a Custom Model using Custom Container Training and a Big Query Dataset. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

# Ensure the following APIs are enabled:
- [BigQuery](https://console.cloud.google.com/apis/library/bigquery.googleapis.com?q=BigQuery)
- [Cloudbuild](https://console.cloud.google.com/apis/library/cloudbuild.googleapis.com?q=Cloudbuild)
- [Container Registry](https://console.cloud.google.com/apis/library/containerregistry.googleapis.com?q=container%20registry)

# Install Vertex SDK for Python


After the SDK installation the kernel will be automatically restarted.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [1]:
MY_PROJECT = "pytorch-tpu-nfs"
MY_STAGING_BUCKET = "gs://automl-samples"  # bucket should be in same region as ucaip

# Copy Big Query Iris Dataset
We will make a Big Query dataset and copy Big Query's public iris table to that dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/iris 

### Make BQ Dataset

In [2]:
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = MY_PROJECT
!bq mk {MY_PROJECT}:ml_datasets

Dataset 'pytorch-tpu-nfs:ml_datasets' successfully created.


### Copy bigquery-public-data.ml_datasets.iris

In [3]:
!bq cp -n bigquery-public-data:ml_datasets.iris {MY_PROJECT}:ml_datasets.iris

Waiting on bqjob_r8adae8c05490196_00000179c0087101_1 ... (0s) Current status: DONE   
Table 'bigquery-public-data:ml_datasets.iris' successfully copied to 'pytorch-tpu-nfs:ml_datasets.iris'


# Create Training Container
We will create a directory and write all of our container build artifacts into that folder.

In [4]:
CONTAINER_ARTIFACTS_DIR = "demo-container-artifacts"

!mkdir {CONTAINER_ARTIFACTS_DIR}

### Create Cloudbuild YAML

In [9]:
cloudbuild_yaml = """steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', 'gcr.io/${PROJECT_ID}/test-custom-container', '.' ]
images: ['gcr.io/${PROJECT_ID}/test-custom-container']"""

with open(f"{CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### Write Dockerfile

In [10]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/Dockerfile

# Specifies base image and tag
FROM gcr.io/google-appengine/python
WORKDIR /root

# Installs additional packages
RUN pip3 install tensorflow tensorflow-io pyarrow

# Copies the trainer code to the docker image.
COPY test_script.py /root/test_script.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "test_script.py"]

Overwriting demo-container-artifacts/Dockerfile


### Write entrypoint script to invoke trainer

In [11]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/test_script.py

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from tensorflow import feature_column
import os

training_data_uri = os.environ["AIP_TRAINING_DATA_URI"]
validation_data_uri = os.environ["AIP_VALIDATION_DATA_URI"]
test_data_uri = os.environ["AIP_TEST_DATA_URI"]
data_format = os.environ["AIP_DATA_FORMAT"]
print("DEBUG_INFO:", training_data_uri,validation_data_uri, test_data_uri, data_format)

def caip_uri_to_fields(uri):
    uri = uri[5:]
    project, dataset, table = uri.split('.')
    return project, dataset, table

feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

target_name = 'species'

def transform_row(row_dict):
  # Trim all string tensors
  trimmed_dict = { column:
                  (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                  for (column,tensor) in row_dict.items()
                  }
  target = trimmed_dict.pop(target_name)

  target_float = tf.cond(tf.equal(tf.strings.strip(target), 'versicolor'), 
                 lambda: tf.constant(1.0),
                 lambda: tf.constant(0.0))
  return (trimmed_dict, target_float)

def read_bigquery(project, dataset, table):
  tensorflow_io_bigquery_client = BigQueryClient()
  read_session = tensorflow_io_bigquery_client.read_session(
      "projects/" + project,
      project, table, dataset,
      feature_names + [target_name],
      [dtypes.float64] * 4 + [dtypes.string],
      requested_streams=2)

  dataset = read_session.parallel_read_rows()
  transformed_ds = dataset.map(transform_row)
  return transformed_ds

BATCH_SIZE = 16

training_ds = read_bigquery(*caip_uri_to_fields(training_data_uri)).shuffle(10).batch(BATCH_SIZE)
eval_ds = read_bigquery(*caip_uri_to_fields(validation_data_uri)).batch(BATCH_SIZE)
test_ds = read_bigquery(*caip_uri_to_fields(test_data_uri)).batch(BATCH_SIZE)

feature_columns = []

# numeric cols
for header in feature_names:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Dense = tf.keras.layers.Dense
model = tf.keras.Sequential(
  [
    feature_layer,
      Dense(16, activation=tf.nn.relu),
      Dense(8, activation=tf.nn.relu),
      Dense(4, activation=tf.nn.relu),
      Dense(1, activation=tf.nn.sigmoid),
  ])

# Compile Keras model
model.compile(
    loss='binary_crossentropy', 
    metrics=['accuracy'],
    optimizer='adam')

model.fit(training_ds, epochs=5, validation_data=eval_ds)

print(model.evaluate(test_ds))

tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

Overwriting demo-container-artifacts/test_script.py


### Build Container

In [12]:
!gcloud builds submit --config {CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml {CONTAINER_ARTIFACTS_DIR}

Creating temporary tarball archive of 3 file(s) totalling 3.2 KiB before compression.
Uploading tarball of [demo-container-artifacts] to [gs://pytorch-tpu-nfs_cloudbuild/source/1622424671.900764-794378dddbbb4a4383b33f0acb23e7ff.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/pytorch-tpu-nfs/builds/4750a3e4-f1dc-493a-82fe-49c3b14e0d3b].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/4750a3e4-f1dc-493a-82fe-49c3b14e0d3b?project=64701051322].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "4750a3e4-f1dc-493a-82fe-49c3b14e0d3b"

FETCHSOURCE
Fetching storage object: gs://pytorch-tpu-nfs_cloudbuild/source/1622424671.900764-794378dddbbb4a4383b33f0acb23e7ff.tgz#1622424672225601
Copying gs://pytorch-tpu-nfs_cloudbuild/source/1622424671.900764-794378dddbbb4a4383b33f0acb23e7ff.tgz#1622424672225601...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 ob

# Run Custom Container Training

## Initialize Vertex SDK for Python

Initialize the *client* for Vertex AI

In [13]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# Create a Managed Tabular Dataset from Big Query Dataset

This section will create a managed Tabular dataset from the iris Big Query table we copied above.

In [14]:
ds = aiplatform.TabularDataset.create(
    display_name="bq_iris_dataset", bq_source=f"bq://{MY_PROJECT}.ml_datasets.iris"
)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/64701051322/locations/us-central1/datasets/1761268094115774464/operations/5289327679009456128
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/64701051322/locations/us-central1/datasets/1761268094115774464
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/64701051322/locations/us-central1/datasets/1761268094115774464')


# Launch a Training Job to Create a Model

We will train a model with the container we built above.

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="train-bq-iris",
    container_uri=f"gcr.io/{MY_PROJECT}/test-custom-container:latest",
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)
model = job.run(
    ds,
    replica_count=1,
    model_display_name="bq-iris-model",
    bigquery_destination=f"bq://{MY_PROJECT}",
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://automl-samples/aiplatform-custom-training-2021-05-31-01:36:10.481 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/144106391982833664?project=64701051322
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/144106391982833664 current state:
PipelineState.PIPELINE_STATE_RUNNING


# Deploy Your Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

# Predict on the Endpoint

In [ ]:
endpoint.predict(
    [{"sepal_length": 5.1, "sepal_width": 2.5, "petal_length": 3.0, "petal_width": 1.1}]
)